In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [2]:
diabetes = pd.read_csv('data/diabetes.csv')

In [3]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = round((diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min()),4)

In [4]:
# binarization float to int
def float_to_binary(x, m, n):
    """Convert the float value `x` to a binary string of length `m + n`
    where the first `m` binary digits are the integer part and the last
    'n' binary digits are the fractional part of `x`.
    """
    x_scaled = round(x * 2 ** n)
    binar = '{:0{}b}'.format(x_scaled, m + n)
    binarized_list = [1 if char == '1' else -1 for char in binar] 
    return binarized_list

In [5]:
for i in diabetes.columns[:-1]:
    diabetes[i] = diabetes[i].apply(lambda x: float_to_binary(x, 1, 7))

In [6]:
# outcome to int
diabetes['Outcome'] = [1 if char == 1.0 else -1 for char in diabetes['Outcome']]

In [7]:
# generate combined data
diabetes['attributes'] = diabetes.drop('Outcome', axis=1).sum(axis=1)

In [8]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes['attributes'], diabetes['Outcome'])

In [9]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [10]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

### Train and Test with Bias

In [ ]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput  = experiment_TRAIN(X_train, 
                                                                                         y_train,
                                                                                         lrParameter=0.2, 
                                                                                         n_epochs=40, 
                                                                                         trainingBias=True,
                                                                                         error_by_epoch = False,
                                                                                         trainingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input'})

In [11]:
with open('error_by_epoch.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [11]:
%%time

weightsEncodingWeight, weightsEncodingInput, weightsHSGS, weightsVectorsClassico, weightVectorsClassicoBin = experiment_TRAIN(X_train, 
                                                                                                             y_train,
                                                                                                             lrParameter=0.2, 
                                                                                                             n_epochs=5, 
                                                                                                             trainingBias=False,
                                                                                                             trainingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input'})


erro HSGS 525.626953125
erro encoding weight 490.5947265625
erro encoding input 514.736328125
erro Classico 490.57705501469906
erro classico Bin 526.1630859375

erro HSGS 526.3720703125
erro encoding weight 490.431640625
erro encoding input 514.2548828125
erro Classico 490.32443950994787
erro classico Bin 526.0048828125

erro HSGS 526.078125
erro encoding weight 489.89453125
erro encoding input 514.529296875
erro Classico 490.2365453741801
erro classico Bin 526.0302734375

erro HSGS 526.0224609375
erro encoding weight 489.98828125
erro encoding input 514.4072265625
erro Classico 490.2365453741801
erro classico Bin 526.0302734375

erro HSGS 525.9638671875
erro encoding weight 490.322265625
erro encoding input 514.615234375
erro Classico 490.2365453741801
erro classico Bin 526.0302734375
CPU times: user 7h 13min 30s, sys: 28.1 s, total: 7h 13min 58s
Wall time: 7h 13min 55s


In [21]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsHSGS, 
          weightsVectorsClassico, 
          weightVectorsClassicoBin]

In [22]:
with open('results/version2/weights_unbiased_float_to_binary.json', 'w+') as fp:
    json.dump(weights, fp)

#### Test

In [23]:
with open('results/version2/weights_unbiased_float_to_binary.json') as json_file:
    weights = json.load(json_file)

In [24]:
from code.experiment_functions import experiment_TEST

In [25]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              weights[4], 
                              repeat=4,
                              bias=False, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input'})

KeyboardInterrupt: 

In [ ]:
with open('results/version2/experiments_weights_unbiased_float_to_binary.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)

In [ ]:
resultsBIAS

{'error_HSGS': [0.6927, 0.6927, 0.6979, 0.7031],
 'error_encoding_weight': [0.6562, 0.6562, 0.6562, 0.6562],
 'error_encoding_input': [0.6771, 0.7031, 0.6875, 0.6927],
 'error_classic': [0.6562, 0.6562, 0.6562, 0.6562],
 'error_classic_bin': [0.7135, 0.7135, 0.7135, 0.7135],
 'weights_learned_HSGS': [[-1,
   1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   1,
   -1,
   1,
   -1,
   1,
   -1,
   1,
   -1,
   1,
   -1,
   -1,
   1,
   1,
   -1,
   1,
   1,
   1,
   -1,
   -1,
   -1,
   1,
   -1,
   -1,
   1,
   1,
   1,
   -1,
   1,
   1,
   -1,
   1,
   1,
   1,
   -1,
   1,
   -1,
   -1,
   1,
   -1,
   -1,
   -1,
   -1,
   1,
   -1,
   -1,
   -1,
   -1,
   1,
   -1,
   1,
   1,
   -1,
   -1,
   1,
   -1],
  [-0.174350161136,
   -0.174350014467,
   0.035497375226,
   0.015595613154,
   0.113491528474,
   0.050273631688,
   -0.1595738272,
   -0.134737998867,
   -0.174350161136,
   0.099904003782,
   0.071404792478,
   0.028088806564,
   -0.157259671432,
   -0.036307251914,
   0.02

In [ ]:
def readAndResults(i):
    with open('results/'+i) as f:
        data = json.load(f)

    erros = ['error_HSGS', 'error_encoding_weight', 'error_encoding_input', 'error_classic', 'error_classic_bin']
    new_data = {'error':[], 'model':[]}

    for erro in erros:
        new_data['error'] = new_data['error'] + data[erro] 
        new_data['model'] = new_data['model'] +  ( (erro+' ') * len(data[erro])).split(' ')[:-1]  

    new_data = pd.DataFrame(new_data)

    new_data.to_csv('results/'+i[:-5]+'_dataframe.csv', index=False)
    
    return new_data

In [ ]:
readAndResults('results/version2/experiments_weights_unbiased_float_to_binary.json')

FileNotFoundError: [Errno 2] No such file or directory: 'results/results/version2/experiments_weights_unbiased_float_to_binary.json'

In [ ]:
df = pd.read_csv('results/version2/experiments_weights_unbiased_float_to_binary_dataframe.csv')

In [ ]:
np.mean(df[df.model == 'error_encoding_weight'])

In [ ]:
np.mean(df[df.model == 'error_encoding_input'])